In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 55.0MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


## Import

In [2]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt, Hannanum, Kkma, Komoran

## Hyperparameter

In [3]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# Hyper-parameters
NUM_LAYERS = 2
D_MODEL = 256 ##word embedding dim
NUM_HEADS = 8 ## D_Model % NUM_HEADS == 0이 되야하므로...
UNITS = 512
DROPOUT = 0.1
EPOCHS = 50
# for data pipelining
BATCH_SIZE = 64
BUFFER_SIZE = 1000

VOCAB_SIZE = 0 # 후에 len(words) 로 바뀜.

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 정규 표현식 필터
RE_FILTER = re.compile("[\"':;~()]")

## Data Load & Preprocessing

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
wear_data = pd.read_csv("/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/csv_file/wear.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
print(customer.shape, store.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.

(15826, 20)
(8381,) (7445,)


In [6]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
             customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

print(len(store_arr))
print(len(customer_arr))
print(store_arr[-1])
print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)

7301
7301
요즘 파스텔 톤이 유행이에요
요즘 유행하는 색깔이 뭐예요?


In [7]:
question = []
answer = []

for Q in customer_arr:
    question.append(Q.replace("[^\w]", " "))

for A in store_arr:
    answer.append(A.replace("[^\w]", " "))

len(question), len(answer)

(7301, 7301)

In [8]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Komoran()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # [\"':;~()] 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [9]:
import time
start = time.time()
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

print("time :", time.time() - start)
print()

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


time : 10.217884540557861

Q : 신발 은 여기 있 는 게 다예 요 ?
A : 네 성인 이나 아동 다 있 어요 발 사이즈 몇 신 으시 시 어요 ?

Q : 230 이 요
A : 편하 게 신 을 수 있 는 거 찾 으시 시 어요 ?

Q : 네 봄 이 니까 편하 게 신 을 수 있 는 거
A : 이런 건 어떻 시 어요 ? 이런 거도 신발 무척 편하 거든요

Q : 굽 좀 높 은 거 없 나요 ?
A : 봄 상품 은 아직 어른 제품 이 많이 안 나오 았 습니다

Q : 언제 들어오 아요 ?
A : 이번 주 지나 면 들어오 ㄹ 거 예 요



In [10]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [11]:
VOCAB_SIZE = len(words)
print("손님과 점원의 말에서 사용된 총 단어의 수 :",len(words))

손님과 점원의 말에서 사용된 총 단어의 수 : 3353


In [12]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [13]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [14]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (신발 은 여기 있는 게 다예 요)
print(question[0])
x_encoder[0]


신발 은 여기 있 는 게 다예 요 ?


array([ 428,  537,  454,  523,  409, 2009,  862, 1016, 2898,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [15]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (<START> 신발 은 여기 있는 게 다예 요)
print(question[0])
x_decoder[0]


신발 은 여기 있 는 게 다예 요 ?


array([   1, 2728,  726, 2267, 1226, 1785,  523,  149, 1316, 2138, 2465,
       3143, 1652, 2875,  149, 2898,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [16]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 입력 출력 (신발 은 여기 있는 게 다예 요 <END>)
print(question[0])
y_decoder[0]


신발 은 여기 있 는 게 다예 요 ?


array([2728,  726, 2267, 1226, 1785,  523,  149, 1316, 2138, 2465, 3143,
       1652, 2875,  149, 2898,    2,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

## Transformer

In [17]:
# decoder inputs use the previous target as input
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': x_encoder,
        'dec_inputs': x_decoder
    },
    {
        'outputs': y_decoder
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [18]:
dataset

<PrefetchDataset shapes: ({inputs: (None, 30), dec_inputs: (None, 30)}, {outputs: (None, 30)}), types: ({inputs: tf.int64, dec_inputs: tf.int64}, {outputs: tf.int64})>

$$\Large{Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

In [19]:
## scaled dot product Attention
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True) # QK^T

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth) #  QK^T / sqrt(d_k)

  if mask is not None:
    logits += (mask * -1e9) # zero padding token softmax 결과가 0이 나오도록
  
  attention_weights = tf.nn.softmax(logits, axis = -1) # softmax(QK^T / sqrt(d_k))

  output = tf.matmul(attention_weights, value) # softmax(QK^T / sqrt(d_k)) * V

  return output

In [20]:
## multi-head attention
## each head need (scaled_dot_product_attention)
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0 # 128,8

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)
  
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(inputs, shape=(batch_size,-1,self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3]) ##????
  
  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    #linear
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    #split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    #scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    #concatenation of heads
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

    #final linear
    outputs = self.dense(concat_attention)

    return outputs

In [21]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [22]:
print(create_padding_mask(tf.constant([[1, 2, 0, 3, 0], [0, 0, 0, 4, 5]])))

tf.Tensor(
[[[[0. 0. 1. 0. 1.]]]


 [[[1. 1. 1. 0. 0.]]]], shape=(2, 1, 1, 5), dtype=float32)


In [23]:
# it handle mask future tokens in a sequence used decoder. and mask pad tokens
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [24]:
print(create_look_ahead_mask(tf.constant([[1, 2, 0, 4, 5]])))

tf.Tensor(
[[[[0. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 1. 0. 1.]
   [0. 0. 1. 0. 0.]]]], shape=(1, 1, 5, 5), dtype=float32)


Positional encoding

since we don't use any rnn, cnn, positional encoding give model position information of words in sentence.

positional encoding vector is added to embedding vector

$$\Large{PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})} $$

In [25]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)
  
  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles #pos/10000^(2i/d_model)

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position = tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model = d_model)
    sines = tf.math.sin(angle_rads[:, 0::2])
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)
  
  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

### Encoder Layer
1. Multi-head attention (with padding mask)
2. 2 dense layers followed by dropout

also has residual connection followd by a layer normalization.

In [26]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query':inputs,
          'key':inputs,
          'value':inputs,
          'mask':padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

### Encoder
1. Input Embedding
2. Positional Encoding
3. `num_layers` encoder layers

Embedding + positional encoding : input

going encoder layers.

output going decoder

In [27]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)#??왜 vocab_size가 들어가지?

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units = units,
        d_model = d_model,
        num_heads = num_heads,
        dropout = dropout,
        name = "encoder_layer_{}".format(i),
    )([outputs, padding_mask])
  
  return tf.keras.Model(
      inputs = [inputs, padding_mask], outputs = outputs, name=name)

### Decoder Layer
1. Masked multi-head attention (with look ahead mask and padding mask)
2. Multi-head attention (with padding mask). `value` and `key` is from encoder output. `query` is from Multi-head attention layer output
3. 2 dense layers followed by dropout

also has residual connection followd by a layer normalization.

In [28]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(shape=(1,None,None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1,1,None), name="padding_mask")

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query' : inputs,
          'key' : inputs,
          'value' : inputs,
          'mask' : look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query' : attention1,
          'key' : enc_outputs,
          'value' : enc_outputs,
          'mask' : padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs = [inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs = outputs,
      name = name)

### Decoder
1. output Embedding
2. Positional Encoding
3. `num_layers` decoder layers

Embedding + positional encoding : input (target)

going decoder layers.

output going final linear layer

In [29]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="decoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(shape=(1,None,None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1,1,None), name='padding_mask')

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units = units,
        d_model = d_model,
        num_heads = num_heads,
        dropout = dropout,
        name = "decoder_layer_{}".format(i),
    )(inputs = [outputs, enc_outputs, look_ahead_mask, padding_mask])
  
  return tf.keras.Model(
      inputs = [inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs = outputs,
      name = name)

### Transformer
1. encoder
2. decoder
3. final linear layer

In [30]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1,1,None),
      name="enc_padding_mask")(inputs)
  
  #mask future tokens for decoder inputs at 1st attention block
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1,None,None),
      name="look_ahead_mask")(dec_inputs)
  
  #mask encoder outputs for the 2nd attention block
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1,1,None),
      name="dec_padding_mask")(inputs)
  
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

## 모델 생성

In [39]:
tf.keras.backend.clear_session()

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

### Loss function
since target sequences are padded, deal this.

In [40]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, max_sequences))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)
  
  mask = tf.cast(tf.not_equal(y_true,0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

### Custom learning rate
use Adam optimizer with custom learning rate
$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$

In [41]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

### Compile Model

In [42]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, max_sequences))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

## Predict_function


In [43]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [44]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [45]:
def evaluate(input_seq):

  input_seq = input_seq.squeeze()
  sentence = tf.expand_dims(input_seq, axis=0)
  output = tf.expand_dims([1], 0)

  for i in range(max_sequences):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, 2):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [46]:
#테스트 데이터셋 준비

test_question_arr = []
test_question = ""

for i in range(14900,wear_data.shape[0]):
        test_question = wear_data.iloc[i].SENTENCE
        test_question_arr.append(test_question)



## Train-50

In [47]:
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
115/115 [==============================] - 4s 34ms/step - loss: 2.3815 - accuracy: 0.0203
Epoch 2/50
115/115 [==============================] - 4s 33ms/step - loss: 2.0287 - accuracy: 0.0356
Epoch 3/50
115/115 [==============================] - 4s 33ms/step - loss: 1.6829 - accuracy: 0.0491
Epoch 4/50
115/115 [==============================] - 4s 32ms/step - loss: 1.4516 - accuracy: 0.0818
Epoch 5/50
115/115 [==============================] - 4s 34ms/step - loss: 1.2860 - accuracy: 0.1052
Epoch 6/50
115/115 [==============================] - 4s 32ms/step - loss: 1.1598 - accuracy: 0.1185
Epoch 7/50
115/115 [==============================] - 4s 32ms/step - loss: 1.0694 - accuracy: 0.1277
Epoch 8/50
115/115 [==============================] - 4s 32ms/step - loss: 1.0010 - accuracy: 0.1344
Epoch 9/50
115/115 [==============================] - 4s 32ms/step - loss: 0.9422 - accuracy: 0.1402
Epoch 10/50
115/115 [==============================] - 4s 32ms/step - loss: 0.8884 - accura

In [48]:
# for train data predict
Komoran_ans_50 = []
for seq_index in range(100):
    Komoran_ans_50.append(convert_index_to_text(evaluate(make_predict_input(question[seq_index])).numpy()[1:],index_to_word))

In [49]:
import pickle

with open('Komoran_ans_50.pkl','wb') as f:
    pickle.dump(Komoran_ans_50, f)

In [50]:
# for train data predict
Komoran_ans_50_test = []
for seq_index in range(0,100):
    Komoran_ans_50_test.append(convert_index_to_text(evaluate(make_predict_input(test_question_arr[seq_index])).numpy()[1:],index_to_word))


In [52]:
with open('Komoran_ans_50_test.pkl','wb') as f:
    pickle.dump(Komoran_ans_50_test, f)

## Train-100

In [53]:
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
115/115 [==============================] - 4s 34ms/step - loss: 0.0725 - accuracy: 0.2906
Epoch 2/50
115/115 [==============================] - 4s 36ms/step - loss: 0.0700 - accuracy: 0.2912
Epoch 3/50
115/115 [==============================] - 4s 36ms/step - loss: 0.0670 - accuracy: 0.2920
Epoch 4/50
115/115 [==============================] - 4s 33ms/step - loss: 0.0649 - accuracy: 0.2924
Epoch 5/50
115/115 [==============================] - 4s 32ms/step - loss: 0.0645 - accuracy: 0.2927
Epoch 6/50
115/115 [==============================] - 4s 33ms/step - loss: 0.0612 - accuracy: 0.2938
Epoch 7/50
115/115 [==============================] - 4s 33ms/step - loss: 0.0586 - accuracy: 0.2946
Epoch 8/50
115/115 [==============================] - 4s 33ms/step - loss: 0.0573 - accuracy: 0.2948
Epoch 9/50
115/115 [==============================] - 4s 32ms/step - loss: 0.0552 - accuracy: 0.2957
Epoch 10/50
115/115 [==============================] - 4s 32ms/step - loss: 0.0542 - accura

In [54]:
# for train data predict
Komoran_ans_100 = []
for seq_index in range(100):
    Komoran_ans_100.append(convert_index_to_text(evaluate(make_predict_input(question[seq_index])).numpy()[1:],index_to_word))

In [55]:
import pickle

with open('Komoran_ans_100.pkl','wb') as f:
    pickle.dump(Komoran_ans_100, f)

In [56]:
# for train data predict
Komoran_ans_100_test = []
for seq_index in range(0,100):
    Komoran_ans_100_test.append(convert_index_to_text(evaluate(make_predict_input(test_question_arr[seq_index])).numpy()[1:],index_to_word))


In [57]:
with open('Komoran_ans_100_test.pkl','wb') as f:
    pickle.dump(Komoran_ans_100_test, f)

In [58]:
Komoran_ans_50

['네 성인 이나 아동 다 있 어요 발 사이즈 몇 신 으시 시 어요 ? ',
 '신발 신어 보 시 겠 어요 ? ',
 '이 게 제일 잘나가 요 ',
 '봄 상품 은 아직 어른 제품 이 많이 안 나오 았 습니다 ',
 '이번 주 지나 면 들어오 ㄹ 거 예 요 ',
 '아니 요 그것 ㄴ 가죽 이 에요 ',
 '2 만 9 천 원 이 에요 ',
 '230 이 없 어요 이것 한 번 신 어 보 시 어요 ',
 '네 ',
 '여기 있 습니다 ',
 '40 % 에 되 는 제품 이예 요 ',
 '가죽 은 없 고 레 자 만 있 어요 ',
 '그 나이 대 분 들 은 이 만원 에 는 저렴 하 ㄴ 걸 ㄴ가요 ? ',
 '네 ',
 '378000 원요 ',
 '네 있 어요 ',
 '휴 무 이 ㅂ니다 ',
 '그 옷 메이커 가방 은 아니 에요 ',
 '그 천 원 이 에요 ',
 '나무 도 있 고 도자기 도 있 어요 ',
 '네 세트 로만 팔 아요 ',
 '그것 ㄴ 16000 원 이 에요 ',
 '세트 에 7 만 원 이 ㅂ니다 ',
 '네 천천히 보 시 어요 ',
 '실크 봄 가을 에 하 죠 ',
 '그 게 털이 좀 더 많이 나가 요 ',
 '네 , 그 가격 이예 요 ',
 '네 물론 이 ㅂ니다 ',
 '네 , 이런 디자인 은 잘 나가 요 ',
 '네 세탁기 로 물 세탁 하 시 면 되 어요 망 에 넣 으면 좋 고요 ',
 '네 있 어요 ',
 '9 시 50 분 이 요 ',
 '아뇨 닫 아요 ',
 '네 왼쪽 에 진열 되 어 있 어요 ',
 '울 ㄹ 샴푸 를 물 에 몇 방울 떨어뜨리 어서 조 물 조 물 하 면 되 ㅂ니다 ',
 '원단 은 구 만 적히 어 있 어요 ',
 '84000 원 에서 20 % 할인 하 면 67000 원 이 요 ',
 '여기 있 는 거 밖에 없 어요 ',
 '헤르메스 이 ㄴ데 이것 도 괜찮 아요 ',
 '네 원단 만 다르 고 디자인 을 똑같이 만들 ㄴ 거 예 요 ',
 '네 , 요즘 은 침 이 에요 ',
 '머리핀 은 밖 에 있 고 브로치 는 안 에 있 습니다 ',
 

In [59]:
Komoran_ans_50_test

['가죽 으로 만들 ㄴ 가방 이 에요 ',
 '예 ',
 '이 제품 은 할인 상품 이 기 때문 에 이렇 게 표시 되 ㄴ 게 다 적히 어 있 어요 ',
 '네 ',
 '그 제품 은 털 이 라 상관없 어요 ',
 '이 제품 은 할인 상품 이 기 때문 에 이렇 게 하 게 6 만 원 이 에요 ',
 '그 옷 은 카드 결제 하 시 어도 되 고 아니 면 환불 은 5 만 원 이 에요 ',
 '그 옷 은 저희 가 가장 인기 가 있 는 것 로 보이 어 드리 ㄹ게요 ',
 '그 상품 은 20 % 세일 하 아서 104000 원 이 ㅂ니다 ',
 '네 , 그 게 가장 많이 나가 고 있 어요 ',
 '그 프 ㄹ 회원 가입 하 시 면 할인 쿠폰 이 발행 되 ㅂ니다 ',
 '네 적립 은 따로 없 어요 . ',
 '네 , 이쪽 으로 오 시 어요 . ',
 '네 , 이쪽 으로 오 시 어요 ',
 '네 이쪽 라인 이 다 미니 백이 에 요 ',
 '할인 코너 는 만원 짜리 이 에요 ',
 '여기 하고 바깥쪽 에 있 어요 ',
 '할인 코너 는 따로 없 어요 ',
 '할인 코너 는 따로 없 어요 ',
 '매주 수요일 과 토요일 에 세일 품목 이 업데이트 되 ㅂ니다 ',
 '네 ',
 '아니 요 원래 총 4 컬러 이 ㄴ데 oo 컬러 하나 가 품절 이 ㅂ니다 ',
 '네 ',
 '저것 십 이만 오천 원 이 에요 ',
 '카드 결제 하 시 면 10 프로 할인 하 아 드리 겠 습니다 ',
 '카드 결제 시 면 10 만 원 에 드리 ㄹ게요 ',
 '저 게 딱 야구 모자 사이즈 예 요 ',
 '네 손님 체인 과 가죽 어느 쪽 으로 보 시 어요 ? ',
 '저 딱 는 프리사 이즈 예 요 ',
 '나이 가 어떻 게 되 시 어요 ? ',
 '요즘 50 분 들 이 이 이 제일 잘나가 ㅂ니다 . ',
 '국산 이 죠 이제 봄 이 면 은 앞 트이 ㄴ 게 안 낫 겠 습니까 ? ',
 '네 , 전부 다 국산 이 에요 이런 거 많이 나가 요 ',
 '여기 서명 부탁 드리 ㅂ니다 ',
 '그럼 이 제품 이 딱 이 ㄴ 거 같

In [60]:
Komoran_ans_100

['네 , 설 만 지나 면 손님 들 이 기다리 시 는 봄 신발 나오 아요 ',
 '편하 게 신 을 수 있 는 거 찾 으시 시 어요 ? ',
 '이런 건 어떻 시 어요 ? ',
 '봄 상품 은 아직 어른 제품 이 많이 안 나오 았 습니다 ',
 '이번 주 지나 면 들어오 ㄹ 거 예 요 ',
 '네 ',
 '2 만 9 천 원 이 ㅂ니다 ',
 '230 이 없 어요 이것 한 번 신 어 보 시 어요 ',
 '네 이건 다 끝나 았 어요 ',
 '여기 있 어요 ',
 '40 % 할인 되 는 제품 이예 요 ',
 '가죽 은 없 고 레 자 만 있 어요 ',
 '5 만 5 천 원 이 ㅂ니다 ',
 '네 ',
 '50 만 원 이 ㅂ니다 ',
 '네 종류 는 있 어요 ',
 '휴 무 이 ㅂ니다 ',
 '네 , 설 쇠 고 나오 아요 ',
 '5 천 원 이 ㅂ니다 ',
 '나무 도 있 고 도자기 도 있 어요 ',
 '네 세트 로만 팔 아요 ',
 '그것 도 5 만 5 천 원요 ',
 '세트 에 7 만 원 이 요 ',
 '네 천천히 보 시 어요 ',
 '실크 봄 가을 에 하 죠 ',
 '그것 도 소 가죽 이 에요 ',
 '네 핸드 메이드 로 하 아 가지 고 그 가격 으로 팔 아요 ',
 '착용 은 안 되 고 살짝 대보 시 면 되 ㅂ니다 ',
 '네 , 조금 씩 들어오 아요 ',
 '끝 부분 이 가죽 이 라서 조심 하 아야 하 아요 ',
 '네 이건 좀 기간 이 라 들어가 ㄴ 데 , 색깔 만 않 아요 ',
 '8 시 까지 하 ㅂ니다 ',
 '아뇨 닫 아요 ',
 '네 여기 는 2 단 이것 는 3 단 제품 있 어요 ',
 '울 ㄹ 샴푸 를 물 에 몇 방울 떨어뜨리 어서 조 물 조 물 조 물 하 면 되 ㅂ니다 ',
 '원단 이 구 김 안 가 고 참 괜찮 아요 ',
 '2 만 원 이 ㅂ니다 ',
 '여기 있 는 거 밖에 없 어요 ',
 '헤르메스 이 ㄴ데 이것 도 괜찮 아요 ',
 '네 원단 만 다르 고 디자인 을 똑같이 만들 ㄴ 사이즈 몇 신 으시 시 어요 ? ',
 '네 , 이 되 ㅂ니다

In [61]:
Komoran_ans_100_test

['예 ',
 '예 ',
 '네 ',
 '네 ',
 '네 핸드 메이드 로 하 아 가지 고 그 가격 어요 ',
 '네 ',
 '네 카드 로 결제 하 시 나요 ? ',
 '그 옷 은 오만 오천 원 이 에요 ',
 '카드 결제 로 결제 하 아 드리 ㄹ게요 ',
 '카드 로 결제 하 시 겠 어요 ? ',
 '하단 에 휴대폰 번호 입력 하 아 주 시 어요 ',
 '전화번호 알리 어 주 시 어요 ',
 '네 잘 나가 ㅂ니다 ',
 '네 잘 나가 잘 나가 네요 ',
 '잘 나가 는 건 다 사이즈 가 빠지 어서 없 네요 이것 230 있 는데 어떻 시 어요 ? ',
 '할인 코너 는 따로 없 어요 ',
 '할인 코너 는 따로 없 어요 ',
 '할인 코너 는 따로 없 어요 ',
 '할인 코너 는 따로 없 어요 ',
 '매주 수요일 과 토요일 에 세일 품목 이 업데이트 되 ㅂ니다 ',
 '내일 은 문 닫 아요 ',
 '매주 수요일 과 토요일 에 세일 품목 이 업데이트 되 ㅂ니다 ',
 '네 ',
 '사 만 오천 원 까지 하 아 드리 ㄹ게요 ',
 '카드 결제 하 시 면 12 만 원 까지 하 아 드리 ㄹ게요 ',
 '카드 로 7 만 5 천 원 까지 하 아 드리 ㄹ게요 ',
 '저 게 딱 야구 모자 사이즈 예 요 ',
 '네 잠시 만 기다리 어 주 시 어요 ',
 '저 게 딱 야구 모자 사이즈 예 요 ',
 '이건 어떻 시 ㄴ가요 ? ',
 '저희 앞 쪽 에 걸리 어 있 는 상품 이 현재 가장 인기 있 는 제품 이 에요 ',
 '국산 이 죠 이제 봄 이 면 은 앞 트이 ㄴ 게 안 낫 겠 습니까 ? ',
 '네 전부 다 국산 은 없 습니다 . ',
 '여기 있 습니다 ',
 '그렇 죠 ',
 '받 으시 는 분 나이 가 어떻 게 되 시 어요 ? ',
 '잠시 만 기다리 어 주 시 어요 ',
 '네 여기 도 있 고 안 에 도 있 어요 ',
 '여기 있 습니다 ',
 '네 , 넣 을 수 있 어요 ',
 '할인 조끼 하 아서 그 가격 이 에요 ',
 '네 그렇 습니다 ',
 '이 게 아주 좋